In [ ]:
import json
import ast
import mysql.connector
from mysql.connector import Error
from utils import *

## DELETE import
import csv

## DELETE END

# defining them as a global variable to be sent.


def get_records(org_id):
    try:
        connection = mysql.connector.connect(
            host="db.avondxva.com",
            database="avondxva",
            user="usharani",
            password="cTKFmaTvXm",
        )
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor(dictionary=True)
            cmd = (
                f"Select * From service Where org_id ={org_id} AND service_end IS NULL;"
            )
            print(f"Executing command: {cmd}")
            cursor.execute(cmd)
            records = cursor.fetchall()
            for record in records:
                phrases = ast.literal_eval(record["phrases"].lower())
                phrases = sorted(phrases, key=len, reverse=True)
                record["pattern"] = create_pattern(phrases)
    except Error as e:
        print("Error while connecting to MySQL", e)
        raise {"error": e}
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")
            return records


def fetch_utterances(csa_id, min_id, max_id):
    """ Fetch all the utterances from utterances table """
    try:
        connection = mysql.connector.connect(
            host="db.avondxva.com",
            database="avondxva",
            user="usharani",
            password="cTKFmaTvXm",
        )
        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)
            # cmd = f"SELECT id, transcript, started_at, ended_at, s3_key FROM utterances_test WHERE csa_id = {csa_id} and date(convert_tz(FROM_UNIXTIME(started_at), 'UTC', 'EST')) = '2021-12-15' ORDER BY started_at ASC"
            cmd = f"SELECT id, transcript, started_at, ended_at, s3_key FROM utterances_test WHERE csa_id = {csa_id} and id >= {min_id} and id <= {max_id} ORDER BY started_at ASC"
            print(f"Executing command: {cmd}")
            cursor.execute(cmd)
            records = cursor.fetchall()
            utterance_data = []
            for record in records:
                transcript = record["transcript"].lower()
                started_at = record["started_at"]
                ended_at = record["ended_at"]
                s3_key = record["s3_key"].lower()
                utterance_data.append([transcript, started_at, ended_at, s3_key])
    except Error as e:
        print("Error while connecting to MySQL", e)
        raise {"error": e}
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")
            return utterance_data


def get_next_engage_id():
    """ Fetch the next engagement id for inserting into Master table """
    # DB query
    """select_query = "SELECT MAX(engagement_id) max_eng FROM AI_Master"
    ai_AI_Master_records = db_obj.execute_select_sql(select_query) """
    try:
        connection = mysql.connector.connect(
            host="db.avondxva.com",
            database="avondxva",
            user="usharani",
            password="cTKFmaTvXm",
        )
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor(dictionary=True)
            cmd = f"SELECT MAX(engagement_id) max_eng FROM AI_Master;"
            print(f"Executing command: {cmd}")
            cursor.execute(cmd)
            records = cursor.fetchall()

            if records[0]["max_eng"] is None or len(records) == 0:
                next_engg_id = 1
            else:
                next_engg_id = int(records[0]["max_eng"]) + 1

    except Error as e:
        print("Error while connecting to MySQL", e)
        raise {"error": e}
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")
            return next_engg_id


def get_next_conv_id():
    """ Fetch the next conversation id for inserting into Master conversation table """

    try:
        connection = mysql.connector.connect(
            host="db.avondxva.com",
            database="avondxva",
            user="usharani",
            password="cTKFmaTvXm",
        )
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor(dictionary=True)
            cmd = f"SELECT MAX(conversation_id) max_conv FROM AI_conv_score_table;"
            print(f"Executing command: {cmd}")
            cursor.execute(cmd)
            records = cursor.fetchall()

            if records[0]["max_conv"] is None or len(records) == 0:
                next_conv_id = 1
            else:
                next_conv_id = int(records[0]["max_conv"]) + 1

    except Error as e:
        print("Error while connecting to MySQL", e)
        raise {"error": e}
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")
            return next_conv_id


def populate_ai_conv_score(engagements, records, org_id, csa_id, store_id):
    try:
        connection = mysql.connector.connect(
            host="db.avondxva.com",
            database="avondxva",
            user="usharani",
            password="cTKFmaTvXm",
        )
        if connection.is_connected():
            cursor = connection.cursor()
            weights = evaluate_engagements(engagements, records)
            single_engagements_weights = weights.copy()
            engagements_list = engagements.copy()
            engagements = str(engagements).replace('"', "")

            for i in range(len(single_engagements_weights)):
                w = single_engagements_weights[i]
                engagement = engagements_list[i]
                query = f"""
                    INSERT INTO AI_conv_score_table (csa_id, store_id, engagement_id, conversation_id, created_at, started_at, ended_at, conversation, score, score_info, s3_url, is_concat, eng_end_flag) 
                                            VALUES ({csa_id}, {store_id}, {str(engagement[0])}, {str(engagement[1])}, CURRENT_TIMESTAMP(), {engagement[3]}, {engagement[4]}, "{engagement[2]}", {w[0]}, "{w[1]}", NULL, 0, {engagement[5]});
                        """
                # "{w[1]}"
                cursor.execute(query)
                connection.commit()

    except Error as e:
        print("Error while connecting to MySQL", e)

        return {"error": e}

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")


def populate_AI_Master_utterance(AI_Master_utterance_data, store_id, csa_id):
    try:
        connection = mysql.connector.connect(
            host="db.avondxva.com",
            database="avondxva",
            user="usharani",
            password="cTKFmaTvXm",
        )
        if connection.is_connected():
            cursor = connection.cursor()

            for utterance in AI_Master_utterance_data:
                query = f"""
                    INSERT INTO AI_Master (csa_id, store_id, engagement_id, conversation_id, created_at, started_at, ended_at, transcript, s3_key) 
                                            VALUES ({csa_id}, {store_id}, {str(utterance[0])}, {str(utterance[1])}, CURRENT_TIMESTAMP(), {utterance[2]}, {utterance[3]}, "{utterance[4]}", "{utterance[5]}");
                        """
                cursor.execute(query)
                connection.commit()

    except Error as e:
        print("Error while connecting to MySQL: ", e)
        raise {"error": e}
        # return {"error": e}

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")


def Update_dates(csa_id, min_id, max_id):
    try:
        connection = mysql.connector.connect(
            host="db.avondxva.com",
            database="avondxva",
            user="usharani",
            password="cTKFmaTvXm",
        )
        if connection.is_connected():
            cursor = connection.cursor()

            query_start = f"""
                update AI_conv_score_table t join
                    (select conversation_id, min(started_at)  as minvalue
                        from AI_Master
                        group by conversation_id
                    ) tmin
                    on t.conversation_id = tmin.conversation_id
                    set started_at = tmin.minvalue;
                    """
            cursor.execute(query_start)
            connection.commit()

            query_end = f"""
               update AI_conv_score_table t join
                    (select conversation_id, max(ended_at)  as max_value
                        from AI_Master
                        group by conversation_id
                    ) tmax
                    on t.conversation_id = tmax.conversation_id
                    set ended_at = tmax.max_value;
                    """
            cursor.execute(query_end)
            connection.commit()

            query_processed = f"""
                update utterances_test set processed = 1 WHERE csa_id = {csa_id} and id >= {min_id} and id <= {max_id}
                    """
            cursor.execute(query_processed)
            connection.commit()
    except Error as e:
        print("Error while connecting to MySQL", e)
        raise {"error": e}
    # return {"error": e}

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")


def lambda_handler(event, context):
    body = json.loads(event["body"])
    org_id = body["org_id"]
    csa_id = body["csa_id"]
    store_id = body["store_id"]
    min_id = body["min_id"]
    max_id = body["max_id"]

    """ org_id = "2"        #body["org_id"]
    csa_id = "9"        #body["csa_id"]
    store_id = "45"      # body["store_id"]5 """

    # Omaha 2
    # store_id_list = ["45"]
    # csa_id_list =[134]

    # Norwalk
    # store_id_list = ["2"]
    # csa_id_list =[ 8, 9 , 10, 11, 12]

    # Milford
    # store_id_list = ["40"]
    # csa_id_list = [119,122,145,146]

    # Omaha 1
    # store_id_list = ["62"]
    # csa_id_list = [336, 337,338,339,340,341,342,343,344,345,347,349]
    """ store_csa = {'2' : [ 8, 9 , 10, 11, 12], '45': [134, 135, 136, 138, 157, 160, 170, 172, 375], '40' : [119,122,145,146], '62' : [336, 337,338,339,340,341,342,343,344,345,347,349]
    , '39' : [332, 376, 377, 385]} """
    # store_csa = {'2': [ 8]}
    """ for key in store_csa: 
        print(key) 
        store_id = key
        csa_id_list = store_csa[key]
        print(csa_id_list)
        for csa_id in csa_id_list:
            records = get_records(org_id)
            utterance_data = fetch_utterances(csa_id)
            next_engg_id = get_next_engage_id()
            conversation_id = get_next_conv_id()
            engagements, AI_Master_utterance_data = get_engagements(utterance_data, next_engg_id, conversation_id)
            populate_ai_conv_score(engagements, records, org_id, csa_id, store_id)
            print("Insertion to AI conversation score table completed")  
            populate_AI_Master_utterance(AI_Master_utterance_data, store_id, csa_id)
            print("Insertion to AI Master table completed")  
            Update_dates(csa_id)
            print("Updated dates") """
    records = get_records(org_id)
    utterance_data = fetch_utterances(csa_id, min_id, max_id)
    next_engg_id = get_next_engage_id()
    conversation_id = get_next_conv_id()
    engagements, AI_Master_utterance_data = get_engagements(
        utterance_data, next_engg_id, conversation_id
    )

    populate_ai_conv_score(engagements, records, org_id, csa_id, store_id)
    print("Insertion to AI conversation score table completed")
    populate_AI_Master_utterance(AI_Master_utterance_data, store_id, csa_id)
    print("Insertion to AI Master table completed")
    Update_dates(csa_id, min_id, max_id)
    print("Updated dates")
    return {
        "statusCode": 200,
        "body": json.dumps({"engagements": engagements}),
    }


# lambda_handler(None, None)  # Testing with None